In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import csv
import pandas as pd
import numpy as np
import tflearn

In [ ]:
def get_dataset(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    df = df[::-1]
    df.columns = ['Siste', 'Kjoper', 'Selger', 'Hoy', 'Lav', 'Totalt_omsatt_NOK', 'Totalt_antall_aksjer_omsatt', 'Antall_off_handler', 'Antall_handler_totalt', 'VWAP']
    return df


def get_currency_data(name):
    df = pd.read_csv(name, sep=';', index_col=0)
    return df


def append_currency_to_stock_price_df(df_stock, df_currency):
    df_stock["Dollarkurs"] = df_currency["1 USD"]


MHG = get_dataset("data\\MHG_data.csv")
nok = get_currency_data("data\\Valutakurser.csv")

append_currency_to_stock_price_df(MHG, nok)
print(MHG)


In [ ]:
def plot_dataframe(df):
    df.plot()
    plt.show()
    

def plot_siste(df):
    df = df.Siste
    df.plot()
    plt.show()

    
def plot_kjoper(df):
    df = df.Kjoper
    df.plot()
    plt.show()
    
    
def plot_omsatt_nok(df):
    df = df.Totalt_omsatt_NOK
    df.plot()
    plt.show()
    
    
def plot_omsatt_antall_aksjer(df):
    df = df.Totalt_antall_aksjer_omsatt
    df.plot()
    plt.show()

def plot_antall_handler_off(df):
    df = df.Antall_off_handler
    df.plot()
    plt.show()
    
    
def plot_antall_handler_totalt(df):
    df = df.Antall_handler_totalt
    df.plot()
    plt.show()


def plot_VWAP(df):
    df = df.VWAP
    df.plot()
    plt.show()

In [ ]:
plot_siste(MHG)
plot_kjoper(MHG)
plot_omsatt_nok(MHG)
plot_omsatt_antall_aksjer(MHG)
plot_antall_handler_off(MHG)
plot_antall_handler_totalt(MHG)
plot_VWAP(MHG)

In [ ]:
# Setup training labels

#print(MHG.Siste)


Up = MHG.Siste.diff(1) > 0

n = len(Up)

train = np.zeros((n, 2))
train[:,0] = (Up.values == 1)
train[:,1] = (Up.values == 0)


cutoff = 1000

Y_train = train[0:cutoff,:]
Y_test = train[cutoff:,:]



days_history = 10
features = 4

input_dimension = features*days_history

Siste_all = np.zeros((n, days_history))
Siste = MHG.Siste.values

Omsatte_all = np.zeros((n, days_history))
Omsatte = MHG.Totalt_antall_aksjer_omsatt.values

Antall_handler_all = np.zeros((n, days_history))
Antall_handler = MHG.Antall_handler_totalt.values

Valutakurs_all = np.zeros((n, days_history))
Valutakurs = MHG.Dollarkurs.values

for i in range(days_history):
    #print(i)
    Siste_all[:,i] = np.roll(Siste, i)
    Omsatte_all[:,i] = np.roll(Omsatte, i)
    Antall_handler_all[:,i] = np.roll(Antall_handler, i)
    Valutakurs_all[:,i] = np.roll(Valutakurs, i)
         
    
#print (Siste_all)
#features = [siste, antall_omsatte, number_of_trades, currency]
X = np.zeros((n, input_dimension))
X[:,0*days_history:(0+1)*days_history] = Siste_all
X[:,1*days_history:(1+1)*days_history] = Omsatte_all
X[:,2*days_history:(2+1)*days_history] = Antall_handler_all
X[:,3*days_history:(3+1)*days_history] = Valutakurs_all

print(X)
X_train = X[0:cutoff,:]
X_test = X[cutoff:,:]

print(X_train)




In [ ]:
print(Y_train.shape)
print(Y_test.shape)
print(X_train.shape)
print(X_test.shape)
print(input_dimension)
import tflearn.datasets.mnist as mnist
#X, Y, testX, testY = mnist.load_data(one_hot=True)
print(X.shape)


In [ ]:


# Build deep neural network
#input_layer = tflearn.input_data(shape=[None, 784])

#dense1 = tflearn.fully_connected(input_layer, 10, activation='tanh',
#                                 regularizer='L2', weight_decay=0.001)

#dropout1 = tflearn.dropout(dense1, 0.8)

#softmax = tflearn.fully_connected(dropout1, 10, activation='softmax')


input_layer = tflearn.input_data(shape=[None, input_dimension])
dense1 = tflearn.fully_connected(input_layer, 10, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 10, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 2, activation='softmax')



# Regression using SGD with learning rate decay and Top-1 accuracy
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
top_k = tflearn.metrics.Top_k(1)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='categorical_crossentropy')





# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X_train, Y_train, n_epoch=20, validation_set=(X_test, Y_test), show_metric=True, run_id="dense_model")

In [ ]:
# -*- coding: utf-8 -*-

""" Deep Neural Network for MNIST dataset classification task.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
"""
from __future__ import division, print_function, absolute_import

import tflearn

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)

# Building deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
dense1 = tflearn.fully_connected(input_layer, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 10, activation='softmax')

# Regression using SGD with learning rate decay and Top-3 accuracy
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
top_k = tflearn.metrics.Top_k(3)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=20, validation_set=(testX, testY),
          show_metric=True, run_id="dense_model")